<a href="https://colab.research.google.com/github/hectoresc/MineriaDeDatos/blob/main/Ejemplo_PatronesSecuenciales_Equipo8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Equipo 8**

**Integrantes:**

Andrea Sarahi Gómez Ávila 2034174

Joshune Juditht Arriaga Gómez 1853668

Héctor Eleazar Escobedo 1745909

Daniel Jair Martinez Ávila 1872137

Base de datos:Groceries dataset

Url:https://www.kaggle.com/heeraldedhia/groceries-dataset

In [ ]:
from google.colab import files
files.upload()

Importar librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
!pip install prefixspan
import copy


In [ ]:
tienda=pd.read_csv("Groceries_dataset.csv")

In [ ]:
tienda.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [ ]:
tienda.shape

(38765, 3)


La base de datos tiene 38765 filas y 3 columnas

# **Limpieza**

In [ ]:
#Revisando si hay valores nulos

In [ ]:
tienda.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

No hay datos nulos

In [ ]:
#Cambiando el formato de fecha
tienda['Date'] = pd.to_datetime(tienda['Date'])


In [ ]:
#Generando columnas para mes y dia

tienda['fecha'] = tienda['Date'].dt.date

tienda['mes'] = tienda['Date'].dt.month
tienda['mes'] = tienda['mes'].replace((1,2,3,4,5,6,7,8,9,10,11,12), 
                                          ('Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto',
                                          'Septiembre','Octubre','Noviembre','Diciembre'))

tienda['dia'] = tienda['Date'].dt.weekday
tienda['dia'] = tienda['dia'].replace((0,1,2,3,4,5,6), 
                                          ('Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo'))
tienda


,Member_number,Date,itemDescription,fecha,mes,dia
0,1808,2015-07-21,tropical fruit,2015-07-21,Julio,Martes
1,2552,2015-05-01,whole milk,2015-05-01,Mayo,Viernes
2,2300,2015-09-19,pip fruit,2015-09-19,Septiembre,Sabado
3,1187,2015-12-12,other vegetables,2015-12-12,Diciembre,Sabado
4,3037,2015-01-02,whole milk,2015-01-02,Enero,Viernes
...,...,...,...,...,...,...
38760,4471,2014-08-10,sliced cheese,2014-08-10,Agosto,Domingo
38761,2022,2014-02-23,candy,2014-02-23,Febrero,Domingo
38762,1097,2014-04-16,cake bar,2014-04-16,Abril,Miercoles
38763,1510,2014-03-12,fruit/vegetable juice,2014-03-12,Marzo,Miercoles


In [ ]:
total_items=len(tienda)
total_dias=len(np.unique(tienda['fecha']))
total_meses=len(np.unique(tienda['Date'].dt.year))
print(total_items,total_dias,total_meses)

38765 728 2


**38765 items vendidos en 728 días durante 24 meses**

In [ ]:
# Quitando la columna Date
tienda.drop('Date', axis = 1, inplace = True)
tienda.head()

,Member_number,itemDescription,fecha,mes,dia
0,1808,tropical fruit,2015-07-21,Julio,Martes
1,2552,whole milk,2015-05-01,Mayo,Viernes
2,2300,pip fruit,2015-09-19,Septiembre,Sabado
3,1187,other vegetables,2015-12-12,Diciembre,Sabado
4,3037,whole milk,2015-01-02,Enero,Viernes


In [ ]:
#Renombrando columnas
tienda.rename(columns = {'itemDescription':'Descripcion','Member_number':'Numero_miembro'}, inplace=True)

In [ ]:
tienda['Descripcion'] = tienda['Descripcion'].str.strip()
tienda['Descripcion'] = tienda['Descripcion'].str.lower()
tienda.head()

,Numero_miembro,Descripcion,fecha,mes,dia
0,1808,tropical fruit,2015-07-21,Julio,Martes
1,2552,whole milk,2015-05-01,Mayo,Viernes
2,2300,pip fruit,2015-09-19,Septiembre,Sabado
3,1187,other vegetables,2015-12-12,Diciembre,Sabado
4,3037,whole milk,2015-01-02,Enero,Viernes


##Secuencias# 

In [ ]:
#Agrupando en una lista los productos que compro un mismo cliente en el mismo dia
tienda=tienda.groupby(['Numero_miembro','fecha'])['Descripcion'].apply(lambda x: list(x))

In [ ]:
tienda.head(30)

Numero_miembro  fecha     
1000            2014-06-24                    [whole milk, pastry, salty snack]
                2015-03-15    [sausage, whole milk, semi-finished bread, yog...
                2015-05-27                           [soda, pickled vegetables]
                2015-07-24                       [canned beer, misc. beverages]
                2015-11-25                          [sausage, hygiene articles]
1001            2014-07-02                    [sausage, whole milk, rolls/buns]
                2014-12-12                                   [whole milk, soda]
                2015-01-20              [frankfurter, soda, whipped/sour cream]
                2015-02-05                                  [frankfurter, curd]
                2015-04-14                                  [beef, white bread]
1002            2014-04-26                                 [butter, whole milk]
                2014-09-02                [frozen vegetables, other vegetables]
             

In [ ]:
tienda.shape

(14963,)

Ahora la base de datos tiene 14963 filas

In [ ]:
#Se convierte el dataset en una lista
transactions = tienda.values.tolist()
transactions[:10]

[['whole milk', 'pastry', 'salty snack'],
 ['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['sausage', 'whole milk', 'rolls/buns'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['frankfurter', 'curd'],
 ['beef', 'white bread']]

Subsecuencias

In [ ]:

def isSubsequence(mainSequence, subSequence):
    subSequenceCopia = list(subSequence) # copiar la secuencia
    return isSubsequenceRecursive(mainSequence, subSequenceCopia) 


def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # Revisa si esta vacío: 
    if (not subSequenceClone):
        return True
    # 
    firstElem = set(subSequenceClone.pop(0))
    # 
    for i in range(start, len(mainSequence)):
        if (set(mainSequence[i]).issuperset(firstElem)):
          
            return isSubsequenceRecursive(mainSequence, subSequenceCopia, i + 1)
    return False

In [ ]:
aSequence = transactions[:1]

In [ ]:
isSubsequence(aSequence, [['pastry', 'salty snack']])

True

In [ ]:
#Calcula la longitud de una secuencia
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)

In [ ]:

print(sequenceLength (transactions[2:3]))

2


## **Prexispan**

In [ ]:
from prefixspan import PrefixSpan


tr = PrefixSpan(transactions)

Subsecuencias con un umbral de soporte minimo de 100

In [ ]:
print(tr.frequent(100))

[(2363, ['whole milk']), (116, ['whole milk', 'yogurt']), (135, ['whole milk', 'rolls/buns']), (115, ['whole milk', 'soda']), (105, ['whole milk', 'other vegetables']), (131, ['whole milk', 'whole milk']), (774, ['pastry']), (281, ['salty snack']), (903, ['sausage']), (121, ['sausage', 'whole milk']), (142, ['semi-finished bread']), (1285, ['yogurt']), (1453, ['soda']), (134, ['pickled vegetables']), (702, ['canned beer']), (236, ['misc. beverages']), (205, ['hygiene articles']), (1646, ['rolls/buns']), (565, ['frankfurter']), (654, ['whipped/sour cream']), (504, ['curd']), (508, ['beef']), (359, ['white bread']), (527, ['butter']), (419, ['frozen vegetables']), (1827, ['other vegetables']), (101, ['other vegetables', 'rolls/buns']), (124, ['other vegetables', 'whole milk']), (1014, ['tropical fruit']), (265, ['sugar']), (263, ['butter milk']), (239, ['specialty chocolate']), (251, ['frozen meals']), (1041, ['root vegetables']), (129, ['detergent']), (102, ['frozen fish']), (734, ['pip

5 subecuencias más frecuentes

In [ ]:
print(tr.topk(5))

[(2363, ['whole milk']), (1827, ['other vegetables']), (1646, ['rolls/buns']), (1453, ['soda']), (1285, ['yogurt'])]
